## Deploy the model to Azure
The goal of the notebook is to deploy the model from Step 4 to Azure. The deployment notebook can be extended to other models. 

The deploy pipelines have the following steps:
1. Build docker image
2. Deploy azure container registry
3. Push docker image to registry
4. Deploy the model in Kubernetes cluster by pulling images from the ACR registry.


## Prerequisites

### Test images
- An image URL is required to test the published Web service.

### Pretrained models
- [Frontal face landmark model](https://github.com/AKSHAYUBHAT/TensorFace/blob/master/openface/models/dlib/shape_predictor_68_face_landmarks.dat) in the same directory as this jupyter notebook.
- [One Eye model](https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml) in the same directory as this jupyter notebook.
- [ResNet152\_ImageNet\_Caffe.model](https://www.cntk.ai/Models/Caffe_Converted/ResNet152_ImageNet_Caffe.model)
- Trained full connected neural network regression model from ***[Step 3. Training_Pipeline](Step3_Training_Pipeline.ipynb)***

All these models are stored in the models folder.


## Write main python file

In [2]:
mainfile = """\

import os
import flask
import numpy as np
from flask import jsonify, request
from model import extract_patches, score_patch, del_cache

app = flask.Flask(__name__)

@app.route('/url/<path:argument>')
def url(argument):
    # create a patch folder
    patch_path = './patches'
    if not os.path.exists(patch_path):
        os.mkdir(patch_path)
    
    # get image url from the query string
    imageURL = request.url.split('=',1)[1]
    
    # extract patches from imageURL
    dimension, face_loc, image_dim = extract_patches(imageURL)
    
    # score each patch
    patch_score= score_patch(patch_path)
    
    # delete the downloaded image and the patches from local
    del_cache(patch_path)
    if os.path.exists('temp.jpg'):
        os.remove('temp.jpg')
    
    data = dict()
    data['patch_score'] = []
    for key in dimension:
        tmp = []
        tmp[:] = dimension[key]
        tmp.append(patch_score[key])
        data['patch_score'].append(tmp)
   
    data['image_score'] = round(np.mean(list(patch_score.values())), 2) 
    data['face_loc'] = face_loc['face_loc']
    data['img_dim'] = image_dim

    return jsonify(patch_score = str(data['patch_score']), image_score = str(data['image_score']), face_loc = str(data['face_loc']), image_dim = str(data['img_dim']))

if __name__ == '__main__':
    app.run(host='0.0.0.0', port = 9580) # port number can be changed in your case

"""

In [3]:
mainpy_path = './main.py'
with open(mainpy_path,'w') as f:
    f.write(mainfile)

## Write docker file

In [4]:
dockerfile = """\

FROM microsoft/cntk:2.5.1-cpu-python3.5
 
RUN dpkg --add-architecture i386 \
&& apt-get update && apt-get install -y \
build-essential \
git \
python3-pip \
python3-tk \
libglib2.0-0:i386 \
&& rm -rf /var/lib/apt/lists/*

RUN pip3 install -U scikit-learn
RUN pip3 install numpy pandas sklearn matplotlib
RUN pip3 install scipy
RUN pip3 install opencv-python-headless
RUN pip3 install dlib
RUN pip3 install scikit-image
RUN pip3 install matplotlib
RUN pip3 install cntk
RUN pip3 install \
flask \
pillow

RUN mkdir /usr/src/nestle
WORKDIR /workspace
RUN chmod -R a+w /workspace
ADD ../models /workspace/models
COPY main.py /workspace
COPY regressionModel.py /workspace
COPY getPatches.py /workspace
COPY model.py /workspace
RUN chmod +x /workspace/main.py
RUN ls /workspace
CMD python3 /workspace/main.py
 
EXPOSE 9580

"""

In [5]:
docker_path = './Dockerfile'
with open(docker_path,'w') as f:
    f.write(dockerfile)

## Build docker image

In [1]:
# Run docker build command to build the nestleapi image.
!sudo docker build -t 'nestleapi' .

In [3]:
# To see the built image, use the docker images command.
!sudo docker images

## ACS parameters

In [8]:
resourceGroupName = '<resource group name>' 
location ='<location>'
acrName = '<acr name>'
aksClusterName = '<cluster name>'

In [19]:
!echo $resourceGroupName
!echo $location
!echo $acrName
!echo $aksClusterName

## Deploy Azure Container Registry for NSH

In [4]:
# login to Azure
!az login

In [5]:
# create resource group
!az group create --name $resourceGroupName --location $location

In [6]:
# create azure container registory under the resource group
!az acr create --name $acrName --resource-group $resourceGroupName  --sku Basic

In [13]:
!sudo az acr login --name $acrName

Login Succeeded


In [7]:
# list acr, resource group, location ......
!az acr list --resource-group $resourceGroupName --output table 

## Push images to registry

In [15]:
# tag the docker image 
!sudo docker tag nestleapi $acrName".azurecr.io/nestleapi:v1"

In [8]:
# Once tagged, run docker images to verify the operation.
!sudo docker images

In [9]:
# Push the nestleapi image to the registry
!sudo docker push $acrName".azurecr.io/nestleapi:v1"

In [18]:
# To verify the above operation
!az acr repository list --name $acrName --output table

Result
---------
nestleapi


## Create Kubernetes cluster

In [10]:
# Create a Kubernetes cluster
!sudo az aks create --resource-group $resourceGroupName --name $aksClusterName --node-count 1 --generate-ssh-keys

In [11]:
# Configure kubectl to connect to your Kubernetes cluster
!sudo az aks get-credentials --resource-group $resourceGroupName --name $aksClusterName

## Configure ACR authentication

Authentication needs to be configured between the AKS cluster and the ACR registry. This involves granting the AKS identity the proper rights to pull images from the ACR registry.

In [22]:
# Get the ID of the service principal configured for AKS.
clientID = !az aks show --resource-group $resourceGroupName --name $aksClusterName --query "servicePrincipalProfile.clientId" --output tsv
clientID = clientID.n

In [23]:
# Get the ACR registry resource ID
arcID = !az acr show --name $acrName --resource-group $resourceGroupName --query "id" --output tsv
arcID = arcID.n

In [12]:
!echo $clientID
!echo $arcID

In [13]:
# Create the role assignment with the values gathered in the last two steps, which grants the proper access 
!az role assignment create --assignee $clientID --role Reader --scope $arcID

## Write yml file

In [26]:
ymlfile = """
apiVersion: apps/v1beta1
kind: Deployment
metadata:
  name: nestleapi
spec:
  replicas: 1
  template:
    metadata:
      labels:
        app: nestleapi
    spec:
      containers:
      - name: nestleapi
        image: {}.azurecr.io/nestleapi:v1
        ports:
        - containerPort: 9580
        resources:
          requests:
            cpu: 250m
          limits:
            cpu: 500m
        env:
        - name: LD_LIBRARY_PATH
          value: "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"

---
apiVersion: v1
kind: Service
metadata:
  name: nestleapi
spec:
  type: LoadBalancer
  ports:
  - port: 80
    targetPort: 9580
  selector:
    app: nestleapi

""".format(acrName)

In [27]:
yml_path = './nestledeploy.yml'
with open(yml_path,'w') as f:
    f.write(ymlfile)

## Deploy application

In [34]:
# Deploy the application to the Kubernetes cluster we created in the last step
!sudo kubectl apply -f nestledeploy.yml

deployment.apps "nestleapi" created
service "nestleapi" created


In [14]:
# check status until STATUS = Running
!sudo kubectl get pods

## Test Application

A Kubernetes service is created which exposes the application to the internet. This process can take a few minutes.

In [17]:
# To monitor progress, use the kubectl get service command with the --watch argument
# Initially the EXTERNAL-IP for the nestleapi service appears as pending. 
# Once the EXTERNAL-IP address has changed from pending to an IP address, use CTRL-C to stop the kubectl watch process.
!sudo kubectl get service nestleapi --watch

In [18]:
# Replace external_ip with the return result from the above
!curl -i -H "Accept: application/json" -H "Content-Type: application/json" http://<external_ip>/url/ImageURL='<image URL>'